## Descripción del conjunto de datos

In [4]:
import pandas as pd  
import os
from IPython.display import display, HTML

# Tamaño inicial de los datos

In [5]:

directory = os.getcwd()
files = os.listdir(directory + '/data')

data = []
for f in files:
    if f != 'DIVERSIFICADOS.csv':
        dataframe = pd.read_csv(directory + '/data/' + f)
        data.append((f, dataframe.shape[0], dataframe.shape[1]))

df = pd.DataFrame(data, columns=['Nombre del archivo', 'Filas', 'Columnas'])
display(HTML(df.to_html(index=False)))

Nombre del archivo,Filas,Columnas
SACATEPEQUEZ.csv,309,18
SUCHITEPEQUEZ.csv,377,18
SAN MARCOS.csv,557,18
PETEN.csv,379,18
CIUDAD CAPITAL.csv,1537,18
CHIQUIMULA.csv,172,18
SANTA ROSA.csv,155,18
IZABAL.csv,360,18
SOLOLA.csv,140,18
RETALHULEU.csv,311,18


# Limpieza

## Listado

Sin duda alguna todas las variables necesitan de limpieza ya que en todas las variables se han encontrado valores nulos.

Se debe de tener cuidado con los nombres de los institutos ya que hay muchas faltas ortográficas.

Se han encontrado un patrón de datos nulos como lo son ***"-"*** que estan de forma consecutiva, strings sin caracteres, Nans, etc en las siguientes variables:

- Establecimiento
- Distrito
- Telefono
- Director
- Supervisor
- Dirección

## Estrategia
- Código: No requiere limpieza.
- Distrito: Sustituir valores en blanco con NA
- Departamento: 
	- Eliminar espacios al inicio y al final.
	- Convertir todo a mayúsculas.
	- Hacer un encoding numérico para facilitar el procesamiento.
- Municipio: 
	- Eliminar espacios al inicio y al final.
	- Convertir todo a mayúsculas.
	- Hacer un encoding numérico.
- Establecimiento, Dirección:
	- Eliminar espacios al inicio y al final.
	- Convertir todo a mayúsculas.
- Teléfono:
	- Tomar en cuenta solo un teléfono cuando se vean dos. `79540830-79540909`.
	- Asegurar que tengan el número correcto de dígitos, de lo contrario es un teléfono inváldo y será reemplazado por `NA`.
- Supervisor, Director:
	- Tomar en cuenta valores como `--` como `NA`.
- Sector, Area, Status, Modalidad, Joranda, Plan:
	- Hacer encoding numérico.
- Nivel: (SARA O Cris)



# Code Book


* **CODIGO** - (Cualitativo ordinal)
	- Descripción
	- Posibles valores
* **DISTRITO** - (Cualitativo nominal)
	- Descripción:
	- Posibles valores: 
* **DEPARTAMENTO** - (Cualitativo nominal)
	- Descripción:
	- Posibles valores: 
	- Encoding:
		- Guatemala - 0
		...
* **MUNICIPIO** - (Cualitativo nominal)
* **ESTABLECIMIENTO** - (Cualitativo nominal)
* **DIRECCIÓN** - (Cualitativo nominal)
* **TELÉFONO** - (Cualitativo nominal)
* **SUPERVISOR** - (Cualitativo nominal)
* **DIRECTOR** - (Cualitativo nominal)
* **NIVEL** - (Cualitativo ordinal)
* **SECTOR** - (Cualitativo nominal)
* **ÁREA** - (Cualitativo nominal)
* **STATUS** - (Cualitativo nominal)
* **MODALIDAD** - (Cualitativo nominal)
* **JORNADA** - (Cualitativo nominal)
* **PLAN** - (Cualitativo nominal)
* **DEPARTAMENTAL** - (Cualitativo nominal)